In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = np.genfromtxt("../data/tng-100-3/cutdata/lates_id.txt")

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_id.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [gas, dm, stars]
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [2]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [3]:
group_cat = physics.properties.galaxy_radius(group_cat, basePath)
group_cat = physics.properties.center_halo(stars, N, group_cat)
for particle in particle_lists:
    particle = physics.properties.relative_pos_radius(particle, N, group_cat)

for i in range(N):
    max_rad = group_cat["SubhaloGalaxyRad"][i]
    for particle in particle_lists:
        particle[i] = particle[i][particle[i]["r"] < max_rad]

print("Calculating stellar masses")
group_cat = physics.properties.total_mass(stars, N, group_cat)
print("Calculating subhalo velocity")
stars, group_cat = physics.properties.subhalo_velocity(stars, N, group_cat)
print("Calculating relative velocities")
stars = physics.properties.relative_velocities(stars, N, group_cat)
print("Calculating half mass radius")
group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")
print("Calculating maximum angular momentum")
group_cat = physics.properties.max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
#print("Rotating subhalos")
stars = physics.geometry.rotate_coordinates(stars, N, group_cat, rot_vector)

#End timer
end = timer()

Calculating stellar masses
Calculating subhalo velocity
Calculating relative velocities
Calculating half mass radius
Calculating maximum angular momentum


In [4]:
print("Galaxy radius")
print(group_cat["SubhaloGalaxyRad"])

Galaxy radius
0     154.633
1      149.16
2     118.384
3     135.639
4     127.379
5     94.9228
6      128.29
7     97.2193
8     100.294
9     83.7265
10    97.2478
11    85.5586
12    82.3664
13     84.557
14    82.6722
15    75.8837
16    67.0623
Name: SubhaloGalaxyRad, dtype: object


In [5]:
print("Masses check")
for i in range(N):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])

Masses check
Group catalogue:  80.1757  Snapshot:  54.72526931762695
Group catalogue:  90.369896  Snapshot:  58.46538543701172
Group catalogue:  43.96099  Snapshot:  30.320581436157227
Group catalogue:  42.83398  Snapshot:  30.461362838745117
Group catalogue:  71.43518  Snapshot:  56.552490234375
Group catalogue:  31.020468  Snapshot:  24.84549331665039
Group catalogue:  61.905563  Snapshot:  46.23890686035156
Group catalogue:  26.348753  Snapshot:  18.92565155029297
Group catalogue:  35.285233  Snapshot:  26.45091438293457
Group catalogue:  26.87838  Snapshot:  18.650514602661133
Group catalogue:  32.6163  Snapshot:  24.920703887939453
Group catalogue:  24.071224  Snapshot:  16.9431095123291
Group catalogue:  23.879963  Snapshot:  17.98649787902832
Group catalogue:  18.338  Snapshot:  14.38700008392334
Group catalogue:  21.333466  Snapshot:  16.593730926513672
Group catalogue:  17.440697  Snapshot:  15.32013988494873
Group catalogue:  19.955704  Snapshot:  14.871585845947266


In [6]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelX"][i], group_cat["SubhaloVelY"][i], group_cat["SubhaloVelZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-171.23763665295385 511.42953563603055 -22.79878854361744
[-175.13785    81.367256   -8.339443]
208.42888417637573 -144.33385048683883 91.88748882556843
[279.23257  -99.84879   82.072495]
103.81072984261439 72.9156542950619 224.41707709524943
[174.64732  57.31744 170.41402]


In [7]:
print("Halfmass radius check")
for i in range(N): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  64.70497  Snapshot:  31.57238065633738
Group catalogue:  79.047134  Snapshot:  36.121992901430765
Group catalogue:  33.925236  Snapshot:  12.658515597823898
Group catalogue:  40.62605  Snapshot:  15.872608862831003
Group catalogue:  39.33135  Snapshot:  27.91179017846851
Group catalogue:  25.502897  Snapshot:  15.98410238078073
Group catalogue:  42.428154  Snapshot:  25.435664895404276
Group catalogue:  39.58687  Snapshot:  20.970284415016593
Group catalogue:  38.12533  Snapshot:  24.30770432358828
Group catalogue:  36.701736  Snapshot:  19.920024057559477
Group catalogue:  48.0739  Snapshot:  22.628384106601988
Group catalogue:  43.243774  Snapshot:  22.62129063967549
Group catalogue:  28.56665  Snapshot:  18.41530676286985
Group catalogue:  20.917725  Snapshot:  12.988256917244314
Group catalogue:  30.294857  Snapshot:  20.849101514509798
Group catalogue:  18.171694  Snapshot:  14.939001622467318
Group catalogue:  37.15208  Snapshot:  24.423606

In [15]:

#Save all fields for particles in one subhalo for later inspection
g = 10
stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(int(indices[g])) +"_stars.pkl")
print(indices[g])

4408.0


In [9]:
#Save group catalogue
#group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [10]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 17 Subhalos: 
17 Seconds
